# Epsilon sample

In [1]:
# source: '/efs/home/archive/prj_bdx/WIPR19DB/dta/'

In [1]:
import pandas as pd
# import numpy as np
# from matplotlib import pyplot as plt
# from sklearn.neighbors import NearestNeighbors
# from sklearn.metrics.pairwise import haversine_distances
# from sklearn.cluster import DBSCAN
# from sklearn import metrics
# from sklearn.neighbors import BallTree
# import math

In [2]:
!dir '/efs/home/archive/prj_bdx/WIPR19DB/dta/'

cluster_labels.dta     ipf_invctry.dta	   isp_citation.dta
famid_apptname.csv     ipf_invgeo.dta	   isp_geo.dta
famid_invname_std.csv  ipf_main.dta	   isp_main.dta
htec_labels.dta        ipf_topcit.dta	   isp_scif.dta
ipf_apptinfo.dta       ipf_totcit.dta	   isp_topcit.dta
ipf_htec.dta	       isp_autcluster.dta  isp_totcit.dta
ipf_invcluster.dta     isp_autctry.dta	   member_states.dta


# Declarations

In [3]:
samplesize = 100000
r_km = 6378.137 # 6371 # multiplier to convert to km (from unit distance)
# 6356.752 km at the poles to 6378.137 km at equator

# World (pats)

In [4]:
# Full dataset main
df = pd.read_stata('/efs/home/archive/prj_bdx/WIPR19DB/dta/ipf_main.dta', index_col='fam_id')  

In [5]:
df = df.drop(columns=['fam_date', 'fam_pct', 'fam_size', 'fam_nr_offices']) 

In [6]:
df.drop(df[(df['fam_year']!=2003) & (df['fam_year']!=2013)].index, inplace=True)

In [7]:
df.head()

,fam_year
fam_id,
1379,2003
1380,2003
1382,2003
1393,2003
1397,2003


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 622769 entries, 1379 to 960016957
Data columns (total 1 columns):
 #   Column    Non-Null Count   Dtype
---  ------    --------------   -----
 0   fam_year  622769 non-null  int16
dtypes: int16(1)
memory usage: 5.9 MB


In [9]:
sys.getsizeof(df)/1024/1024

5.939203262329102

In [10]:
# Full dataset inventors
dfinv = pd.read_stata('/efs/home/archive/prj_bdx/WIPR19DB/dta/ipf_invgeo.dta', index_col='fam_id')  

In [11]:
dfinv.head()

,invgeo_lat,invgeo_lng,invgeo_wgt
fam_id,,,
58,49.8647,8.66091,0.500000
58,49.9679,8.66340,0.500000
76,50.8146,3.05979,0.500000
76,50.9488,3.27389,0.500000
98,53.5773,9.96754,0.333333


In [12]:
df = pd.merge(df,dfinv, on='fam_id')

In [13]:
del dfinv

In [14]:
# Full dataset main
dftec = pd.read_stata('/efs/home/archive/prj_bdx/WIPR19DB/dta/ipf_htec.dta', index_col='fam_id') 
dftec.head()

,htec_id,htec_wgt
fam_id,,
58,1306,0.500000
76,4231,0.333333
98,2110,1.000000
104,1408,0.500000
112,5233,1.000000


In [15]:
df = pd.merge(df,dftec, on='fam_id')
df.head()

,fam_year,invgeo_lat,invgeo_lng,invgeo_wgt,htec_id,htec_wgt
fam_id,,,,,,
1379,2003,48.80531,2.33275,0.333333,4227,0.5
1379,2003,48.80531,2.33275,0.333333,4230,0.5
1379,2003,51.38780,5.44320,0.333333,4227,0.5
1379,2003,51.38780,5.44320,0.333333,4230,0.5
1379,2003,51.44160,5.46972,0.333333,4227,0.5


In [16]:
df['wgt']=df['invgeo_wgt']*df['htec_wgt']
df.head()

,fam_year,invgeo_lat,invgeo_lng,invgeo_wgt,htec_id,htec_wgt,wgt
fam_id,,,,,,,
1379,2003,48.80531,2.33275,0.333333,4227,0.5,0.166667
1379,2003,48.80531,2.33275,0.333333,4230,0.5,0.166667
1379,2003,51.38780,5.44320,0.333333,4227,0.5,0.166667
1379,2003,51.38780,5.44320,0.333333,4230,0.5,0.166667
1379,2003,51.44160,5.46972,0.333333,4227,0.5,0.166667


In [17]:
df = df.drop(columns=['invgeo_wgt', 'htec_wgt']) 

In [18]:
df.rename(columns={'fam_year': 'year', 'invgeo_lat': 'lat', 'invgeo_lng': 'lng','htec_id':'tec'}, inplace=True)

In [19]:
df.head()

,year,lat,lng,tec,wgt
fam_id,,,,,
1379,2003,48.80531,2.33275,4227,0.166667
1379,2003,48.80531,2.33275,4230,0.166667
1379,2003,51.38780,5.44320,4227,0.166667
1379,2003,51.38780,5.44320,4230,0.166667
1379,2003,51.44160,5.46972,4227,0.166667


In [20]:
del dftec

In [21]:
df.groupby(['year'])['wgt'].sum()

year
2003    247917.0
2013    358170.0
Name: wgt, dtype: float32

In [22]:
print('before:', df.wgt.sum())
df = df.groupby(['year','lat','lng','tec'],as_index=False).sum()
print('after:', df.wgt.sum())

before: 606086.75
after: 606087.25


In [23]:
df.to_csv('ipf0313.csv', index=False)

In [24]:
df.head()

,year,lat,lng,tec,wgt
0,2003,-46.40000,168.35001,1409,0.250000
1,2003,-46.40000,168.35001,3220,0.250000
2,2003,-46.23148,169.75117,3429,0.500000
3,2003,-46.23148,169.75117,4125,0.500000
4,2003,-46.13330,167.68300,4227,0.166667


In [28]:
df['tec2'] = df['tec'].astype(str).str[0:2]
df

,year,lat,lng,tec,wgt,tec2
0,2003,-46.40000,168.35001,1409,0.250000,14
1,2003,-46.40000,168.35001,3220,0.250000,32
2,2003,-46.23148,169.75117,3429,0.500000,34
3,2003,-46.23148,169.75117,4125,0.500000,41
4,2003,-46.13330,167.68300,4227,0.166667,42
...,...,...,...,...,...,...
978051,2013,69.75000,-128.86700,4227,0.500000,42
978052,2013,69.92139,23.27927,5135,0.083333,51
978053,2013,70.25330,-148.35001,2110,0.083333,21
978054,2013,70.25330,-148.35001,3419,0.083333,34


In [29]:
df=df.drop(columns=['tec'])
df.head()

,year,lat,lng,wgt,tec2
0,2003,-46.40000,168.35001,0.250000,14
1,2003,-46.40000,168.35001,0.250000,32
2,2003,-46.23148,169.75117,0.500000,34
3,2003,-46.23148,169.75117,0.500000,41
4,2003,-46.13330,167.68300,0.166667,42


In [30]:
print('before:', df.wgt.sum())
df = df.groupby(['year','lat','lng','tec2'],as_index=False).sum()
print('after:', df.wgt.sum())

before: 606087.25
after: 606087.06


In [31]:
df.to_csv('ipf0313tec2.csv', index=False)

# World (pubs)

In [10]:
!dir '/efs/home/archive/prj_bdx/WIPR19DB/dta/'

cluster_labels.dta     ipf_invctry.dta	   isp_citation.dta
famid_apptname.csv     ipf_invgeo.dta	   isp_geo.dta
famid_invname_std.csv  ipf_main.dta	   isp_main.dta
htec_labels.dta        ipf_topcit.dta	   isp_scif.dta
ipf_apptinfo.dta       ipf_totcit.dta	   isp_topcit.dta
ipf_htec.dta	       isp_autcluster.dta  isp_totcit.dta
ipf_invcluster.dta     isp_autctry.dta	   member_states.dta


In [3]:
# Full dataset main
df = pd.read_stata('/efs/home/archive/prj_bdx/WIPR19DB/dta/isp_main.dta', index_col='isp_id')  

In [5]:
df = df.drop(columns=['pubdate']) 

In [6]:
df.drop(df[(df['pubyear']!=2003) & (df['pubyear']!=2013)].index, inplace=True)

In [7]:
df.head()

,pubyear
isp_id,
21174295,2013
21174296,2013
21174297,2013
21174298,2013
21174299,2013


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2460708 entries, 21174295 to 42860319
Data columns (total 1 columns):
 #   Column   Dtype
---  ------   -----
 0   pubyear  int16
dtypes: int16(1)
memory usage: 23.5 MB


In [9]:
sys.getsizeof(df)/1024/1024

23.46715545654297

In [11]:
# Full dataset inventors
dfinv = pd.read_stata('/efs/home/archive/prj_bdx/WIPR19DB/dta/isp_geo.dta', index_col='isp_id')  

In [13]:
dfinv = dfinv.drop(columns=['autgeo_q']) 
dfinv.head()

,lat,lng,autgeo_wgt
isp_id,,,
1,35.201698,-89.971497,1.000000
3,38.951698,-92.334099,1.000000
5,38.986301,-3.929100,1.000000
6,34.053101,-118.380600,0.166667
6,34.147800,-118.144501,0.166667


In [14]:
df = pd.merge(df,dfinv, on='isp_id')

In [15]:
del dfinv

In [16]:
# Full dataset main
dftec = pd.read_stata('/efs/home/archive/prj_bdx/WIPR19DB/dta/isp_scif.dta', index_col='isp_id') 
dftec.head()

,scif_id,scif_wgt
isp_id,,
1,8,0.5
1,4,0.5
3,7,0.5
3,2,0.5
5,12,1.0


In [17]:
df = pd.merge(df,dftec, on='isp_id')
df.head()

,pubyear,lat,lng,autgeo_wgt,scif_id,scif_wgt
isp_id,,,,,,
21174295,2013,48.536900,9.053600,1.000000,2,0.5
21174295,2013,48.536900,9.053600,1.000000,8,0.5
21174296,2013,39.919868,32.854271,0.333333,8,1.0
21174296,2013,39.966702,32.922501,0.666667,8,1.0
21174297,2013,38.003632,112.448883,1.000000,3,0.5


In [18]:
df['wgt']=df['autgeo_wgt']*df['scif_wgt']
df.head()

,pubyear,lat,lng,autgeo_wgt,scif_id,scif_wgt,wgt
isp_id,,,,,,,
21174295,2013,48.536900,9.053600,1.000000,2,0.5,0.500000
21174295,2013,48.536900,9.053600,1.000000,8,0.5,0.500000
21174296,2013,39.919868,32.854271,0.333333,8,1.0,0.333333
21174296,2013,39.966702,32.922501,0.666667,8,1.0,0.666667
21174297,2013,38.003632,112.448883,1.000000,3,0.5,0.500000


In [19]:
df = df.drop(columns=['autgeo_wgt', 'scif_wgt']) 

In [20]:
df.rename(columns={'pubyear': 'year', 'scif_id':'tec'}, inplace=True)

In [21]:
df.head()

,year,lat,lng,tec,wgt
isp_id,,,,,
21174295,2013,48.536900,9.053600,2,0.500000
21174295,2013,48.536900,9.053600,8,0.500000
21174296,2013,39.919868,32.854271,8,0.333333
21174296,2013,39.966702,32.922501,8,0.666667
21174297,2013,38.003632,112.448883,3,0.500000


In [22]:
del dftec

In [23]:
df.groupby(['year'])['wgt'].sum()

year
2003     889953.0
2013    1493694.0
Name: wgt, dtype: float32

In [24]:
print('before:', df.wgt.sum())
df = df.groupby(['year','lat','lng','tec'],as_index=False).sum()
print('after:', df.wgt.sum())

before: 2383647.2
after: 2383646.8


In [25]:
df.to_csv('isp0313.csv', index=False)